In [2]:
[(True, 'b' + i) if i!='--' else (True, '--') for i in ['--', 'R', 'Q', 'N', 'B']]

[(True, '--'), (True, 'bR'), (True, 'bQ'), (True, 'bN'), (True, 'bB')]

In [ ]:
def main():
    p.init()
    screen = p.display.set_mode((WIDTH, HEIGHT))
    clock = p.time.Clock()
    screen.fill(p.Color("white"))
    pawn_promotion_scene = None

    game = ChessEngine.GameState() #we initialize game as our primary object
    valid_moves: List[ChessEngine.Move] = game.get_valid_moves() #these are moves possible at the very beginning
    move_made: bool = False #This little exit_flag logic is used to avoid generating moves after every loop, saving time
    #print(game.board)

    #load_images() #actually load the images 
    draw_game_state(screen, game) #draw initial board config
    
    #this loops basically keeps the game running, it handles events happening or whatever
    running = True
    sq_selected: Union[Tuple[int, int], Tuple[()]] = () #keeps track of the last click of the user
    player_clicks: List[Tuple] = [] #keeps track of couples of players input. Basically is needed to update pieces inside of the board

    while(running):
        for e in p.event.get():
            if e.type == p.QUIT:
                running = False

            ## MOUSE HANDLER
            if e.type == p.MOUSEBUTTONDOWN:
                location = p.mouse.get_pos() 
                col, row = location[0]//SQ_SIZE, location[1]//SQ_SIZE 
                
                if sq_selected == (row,col):
                    sq_selected = () #we deselect if the same click is given as an input twice
                    player_clicks = []
                
                else:
                    sq_selected = (row, col)
                    player_clicks.append(sq_selected) #record the values

                if (len(player_clicks) == 1):
                    if game.board[row, col] == '--':
                        sq_selected = ()
                        player_clicks = []

                #update with the move if necessary, we move after the second input
                if len(player_clicks) == 2:
                    move = ChessEngine.Move(player_clicks[0], player_clicks[1], game)
                    
                    for engine_move in valid_moves:
                        if move == engine_move:
                            if engine_move.pawn_promotion[0]:
                                engine_move.pawn_promotion = (True, handle_pawn_promotion(screen, game, engine_move)) # we first want to handle pawn promotion separately in order to interact correctly with the game

                            game.make_move(engine_move)
                            print(engine_move)
                            move_made = True
                            sq_selected = ()
                            player_clicks = []

                            break

                    else:
                        player_clicks = [sq_selected]

            ## KEYBOARD EVENTS HANDLER
            if e.type == p.KEYDOWN:
                if e.key == p.K_z:
                    game.undo_move()
                    move_made = True
                    sq_selected = ()
                    player_clicks = []

        if move_made:
            valid_moves = game.get_valid_moves()
            move_made = False
        
        draw_game_state(screen, game, highlighted=(sq_selected), moves=valid_moves)
        clock.tick(MAX_FPS)
        p.display.flip()
##



##############################
### GRAPHICS FEATURES CODE ###
##############################


def draw_game_state(screen: p.Surface, gs: ChessEngine.GameState, highlighted: Optional[Union[Tuple[int, int], Tuple[()]]] = None, moves=None) -> None:
    '''
    Method that simply draws things down in a dynamical manner. Relies on the two helpers method cited below. It returns nothing, it just edits current status of the display provided as parameter.
    '''
    draw_board(screen, gs, highlighted, moves) #we first draw the skeleton
    draw_pieces(screen, gs) #and we place the pieces on top of it
    pass
##

def draw_board(screen: p.Surface, game: ChessEngine.GameState, highlighted: Optional[Union[Tuple[int, int], Tuple[()]]], moves: Optional[List[ChessEngine.Move]] = None) -> None:
    '''Handles the drawing board squares (basically the skeleton)'''

    possible_reaching_squares = [(move.end_row, move.end_col) for move in moves if (move.start_row, move.start_col)==highlighted] if moves is not None else []

    check_square: Tuple[int, int] = (-1,-1)
    if game.in_check:
        check_square = game.white_king_pos if game.white_to_move else game.black_king_pos

    for r in range(DIMENSION):
        for c in range(DIMENSION):
            color = GRAPHICS_PALETTE[((r+c)%2)]
            HIGHLIGHT_COLOR = LIGHT_HIGHLIGHT_COLOR[((r+c)%2)]                

            # highlighted square
            if (r,c) == highlighted: 
                col_to_move = 'w' if game.white_to_move else 'b'
                if game.board[r,c][0] == col_to_move: #if selected square is one of the pieces to move
                    p.draw.rect(screen, HIGHLIGHT_COLOR, p.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))
                else:
                    p.draw.rect(screen, color, p.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))

            # possible moves squares
            elif (r,c) in possible_reaching_squares:
                p.draw.rect(screen, HIGHLIGHT_COLOR, p.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))

            elif (r,c) == check_square:
                p.draw.rect(screen, CHECK_HIGHLIGHT_COLOR, p.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))

            # all the remainder
            else:
                p.draw.rect(screen, color, p.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))
    pass
##

def draw_pieces(screen: p.Surface, gs: ChessEngine.GameState) -> None:
    '''
    Draw the pieces on the board based on current GameState.board object passed as parameter.
    '''
    for r in range(DIMENSION):
        for c in range(DIMENSION):
            piece = gs.board[r,c]
            if piece != '--': #if the cell is not empty
                #screen.blit(IMAGES[piece], p.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))
                pass
##


#this is suboptimal (we open different loops, bad environment)
def handle_pawn_promotion(screen: p.Surface, game: ChessEngine.GameState, move: ChessEngine.Move) -> Literal['--', 'wQ', 'wR', 'wN', 'wB', 'bQ', 'bR', 'bN', 'bB']:
    
    pieces = ['Q', 'R', 'B', 'N']
    promotion_figures: List[Literal['--', 'wQ', 'wR', 'wN', 'wB', 'bQ', 'bR', 'bN', 'bB']] = [game.col() + i for i in pieces]  # pyright: ignore[reportAssignmentType]

    p.draw.rect(screen, 'black', p.Rect(BOX_INIT_X, BOX_INIT_Y, BOX_WIDTH, BOX_HEIGHT))

    for i, piece in enumerate(promotion_figures):
         p.draw.rect(screen, GRAPHICS_PALETTE[i%2], p.Rect(PAWN_PROMOTION_INIT_X + i*PAWN_PROMOTION_HEIGHT, PAWN_PROMOTION_INIT_Y, PAWN_PROMOTION_HEIGHT, PAWN_PROMOTION_HEIGHT))

         #screen.blit(IMAGES[piece], p.Rect(PAWN_PROMOTION_INIT_X + i*PAWN_PROMOTION_HEIGHT, PAWN_PROMOTION_INIT_Y, PAWN_PROMOTION_HEIGHT, PAWN_PROMOTION_HEIGHT))

    p.display.update()

    # handling the exit event
    exit_request: bool = False
    while not exit_request:
        for e in p.event.get():
            if e.type == p.MOUSEBUTTONDOWN:
                loc = p.mouse.get_pos()
                x, y = loc[0], loc[1]

                x = loc[0] - PAWN_PROMOTION_INIT_X
                y = loc[1] - PAWN_PROMOTION_INIT_Y

                if(all([0<=x<=PAWN_PROMOTION_WIDTH, 0<=y<=PAWN_PROMOTION_HEIGHT])):
                    x:int = x//PAWN_PROMOTION_HEIGHT
                    return promotion_figures[x]
                    exit_request = not exit_request
##

In [1]:
s = 'a'
s += 'b' if True else 'c'

print(s)

ab


In [3]:
for num, char in enumerate('abcdefgh'):
    print(f'{num}: {char}')

0: a
1: b
2: c
3: d
4: e
5: f
6: g
7: h


In [27]:
import numpy as np
a = []
a += 64 * ['--']
a[8:16] = 8*['wP']

np.array([i if i else '--' for i in a]).reshape((8,8))


array([['--', '--', '--', '--', '--', '--', '--', '--'],
       ['wP', 'wP', 'wP', 'wP', 'wP', 'wP', 'wP', 'wP'],
       ['--', '--', '--', '--', '--', '--', '--', '--'],
       ['--', '--', '--', '--', '--', '--', '--', '--'],
       ['--', '--', '--', '--', '--', '--', '--', '--'],
       ['--', '--', '--', '--', '--', '--', '--', '--'],
       ['--', '--', '--', '--', '--', '--', '--', '--'],
       ['--', '--', '--', '--', '--', '--', '--', '--']], dtype='<U2')

In [18]:
['aa'] + 8 * ['--']

['aa', '--', '--', '--', '--', '--', '--', '--', '--']

In [12]:
i = 'b'
white_to_move = (i=='w')
print(white_to_move)

False
